In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    print(f'======== {version_str} ========')
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)

    from section_utils import SectionUtilities
    su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)

    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    lru.build_isheader_logistic_regression_elements()
    lru.build_pos_logistic_regression_elements()

    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(e.__class__)

======== Neo4j/4.4.7 ========
Got this <class 'numpy.core._exceptions._ArrayMemoryError'> error in build_pos_logistic_regression_elements trying  to turn the pos_symbol TF-IDF matrix into a normal array: Unable to allocate 352. MiB for an array with shape (968, 47602) and data type float64


In [5]:

import os
from datetime import datetime
from urllib.parse import urlparse, parse_qs
from urllib.error import HTTPError, URLError
from IPython.display import clear_output
import random
import requests
import time
print(f'Last run on {datetime.now()}')

Last run on 2022-08-19 09:52:48.817390



----

In [9]:

driver = wsu.get_driver()
wsu.log_into_dice(driver, verbose=False)
file_path = '../data/html/dice_email.html'
page_soup = wsu.get_page_soup(file_path)
tags_list = ['#jobDescriptionText', 'table', 'tbody', 'tr', 'td', 'div', 'div', 'div', 'div', 'div', 'div',
             'div', 'div', 'div', 'table', 'tbody', 'tr:nth-child(1)', 'td', 'table',
             'tbody', 'tr:nth-child(1)', 'td', 'a']
css_selector = ' > '.join(tags_list)
link_soups_list = page_soup.select(css_selector)
url_strs_set = set()
for link_soup in link_soups_list:
    url_str = link_soup['href']
    url_strs_set.add(url_str)
len(url_strs_set)

Getting the FireFox driver


5

In [10]:

files_list = []
for url_str in url_strs_set:
    wsu.driver_get_url(driver, url_str, verbose=False)
    time.sleep(3)
    file_node_dict, files_list = su.load_dice_posting_url(url_str, driver, files_list, verbose=False)
print(f'Fileing {len(files_list)} postings complete')

Saving to ../saves/html\Software_Development_Test_Engineer_SDET_BayOne_Solutions_Remote.html
Fileing 1 postings complete


In [11]:

driver.close()
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    if os.path.isfile(file_path):
        page_soup = wsu.get_page_soup(file_path)
        for div_soup in page_soup.find_all(name='div', attrs={'id': 'jobDescriptionText'}):
            child_strs_list = ha.get_navigable_children(div_soup, [])
            assert child_strs_list, f'Something is wrong with {file_name}'
            cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
files_list

['Software_Development_Test_Engineer_SDET_BayOne_Solutions_Remote.html']